In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# %load_ext cudf.pandas
%config Completer.use_jedi = False



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s5e3/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e3/test.csv')

In [ ]:
test['winddirection'] = test['winddirection'].fillna(test['winddirection'].median())
test.isna().sum()

In [ ]:
train.describe()

In [ ]:
x = ['day', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection',
       'windspeed']
y = ['rainfall']

x_train_data = train[x]
y_train_data = train[y]

test_df = test[x]

n_neighbors = 3
random_state = 0

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x_train_data, y_train_data, test_size=0.2, random_state=random_state)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

max_output_score, max_output_df = float('-inf'), None

models = [
    ('LoR', LogisticRegression()),
    ('GBC', GradientBoostingClassifier(
        n_estimators = 500,
        max_depth = 7,
        min_samples_split = 15,
        min_samples_leaf = 10
    )),
    ('HBC', HistGradientBoostingClassifier(l2_regularization = 0.01,
                                             early_stopping = False,
                                             learning_rate = 0.001,
                                             max_iter = 10000,
                                             max_depth = 5,
                                             max_bins = 255,
                                             min_samples_leaf = 15,
                                             max_leaf_nodes = 10)),
    ('RFC', RandomForestClassifier(n_estimators = 500, 
                                   max_depth = 7,
                                   min_samples_split = 15,
                                   min_samples_leaf = 10)),
    ('DTC', DecisionTreeClassifier()),
    ('ETC', ExtraTreesClassifier(n_estimators = 500, 
                                 max_depth = 7,
                                 min_samples_split = 15,
                                 min_samples_leaf = 10)),
    ('XGBC', XGBClassifier(objective = 'binary:logistic',
                           tree_method = 'hist',
                           colsample_bytree = 0.7, 
                           gamma = 2, 
                           learning_rate = 0.01, 
                           max_depth = 7, 
                           min_child_weight = 10, 
                           n_estimators = 500, 
                           subsample = 0.7)),
    ('LGBC', LGBMClassifier(objective = 'binary',
                             n_estimators = 500,
                             max_depth = 7,
                             learning_rate = 0.01,
                             num_leaves = 20,
                             reg_alpha = 3,
                             reg_lambda = 3,
                             subsample = 0.7,
                             colsample_bytree = 0.7)),
    ('CBC', CatBoostClassifier(loss_function = 'Logloss',
                                iterations = 500,
                                learning_rate = 0.01,
                                depth = 7,
                                random_strength = 0.5,
                                bagging_temperature = 0.7,
                                border_count = 30,
                                l2_leaf_reg = 5,
                                verbose = False, 
                                task_type = 'CPU'))
]

for name, model in models:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    accuracy_score = accuracy_score(y_test, y_pred)
    print('-'*45, name, '-'*45)
    print(f"{name}: Mean Squared Error (MSE): {mse:.4f}")
    print(f'{name}: accuracy_score :{accuracy_score(y_test, y_pred):.4f}')
    print('-'*100)
    output = model.predict(test_df)
    arr = test.loc[:,'id']
    output_df = pd.DataFrame({'id':arr, 'defects': output.flatten()})
    output_df.to_csv(f'/kaggle/working/{name}-output.csv', encoding='utf-8', index=False)

    if max_output_score < accuracy_score:
            max_output_score, max_output_df = accuracy_score, output_df.copy()


print('Final output store in submission.csv')
max_output_df.to_csv(f'/kaggle/working/submission.csv', encoding='utf-8', index=False)

In [ ]:
test['winddirection'].median()